In [12]:
import random
class Agent() :
    s = 11
    def shouldJump(self, v) :
        print(v)
        return random.random() < 0.1
def nextAgent() :
    return Agent()

In [13]:
import FlappyBird.customFlappy 

In [14]:
FlappyBird.customFlappy.main(nextAgent)

error: Couldn't open assets/sprites/0.png